In [77]:
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number
import seaborn as sb
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud, STOPWORDS
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import svm
import math
from nltk.collocations import *
from nltk import word_tokenize
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import cross_validate
from sklearn.metrics import *

In [41]:
df_train = pd.read_csv('../data/train_set.tsv', sep='\t')

df_test = pd.read_csv('../data/test_set.tsv', sep='\t')



In [42]:
df_train

,Id,Title,Content,Category
0,1034,"Rover deal 'may cost 2,000 jobs'","Some 2,000 jobs at MG Rover's Midlands plant m...",business
1,1008,India calls for fair trade rules,"India, which attends the G7 meeting of seven l...",business
2,5011,UK net users leading TV downloads,British TV viewers lead the trend of illegally...,tech
3,2034,Vera Drake scoops film award,Oscar hopefuls Mike Leigh and Imelda Staunton ...,entertainment
4,5172,Gadget show heralds MP3 Christmas,Partners of those who love their hi-tech gear ...,tech
...,...,...,...,...
1775,5265,Half of UK's mobiles 'go online',Multimedia mobile phones are finally showing s...,tech
1776,2213,TV show unites Angolan families,Angolan families who are attempting to track e...,entertainment
1777,2214,Volcano drama erupts on BBC One,"Supervolcano, a docu-drama about a volcanic er...",entertainment
1778,4322,Scotland v Italy (Sat),"Murrayfield, EdinburghSaturday, 26 February140...",sport


# Wordclouds

In [ ]:
cat_df = df_train.groupby('Category')['Content'].apply(lambda x: ''.join(x))


In [ ]:
text = cat_df['business']

wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'gray',
    collocations=False,
    stopwords = STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
text = cat_df['entertainment']

wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'gray',
    collocations=False,
    stopwords = STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
text = cat_df['politics']

wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'gray',
    collocations=False,
    stopwords = STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
text = cat_df['sport']

wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'gray',
    collocations=False,
    stopwords = STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
text = cat_df['tech']

wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'gray',
    collocations=False,
    stopwords = STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

# Classification

### Tokenization of all articles' content, to use in classification methods

#### *Using Bag of Words*

In [23]:
vectorizer = CountVectorizer()

In [24]:
tokens = df_train.Content.values
X = vectorizer.fit_transform(tokens)
vectorizer.get_feature_names()
X = X.toarray()

In [25]:
test_tokens = df_test.Content.values
X_test = vectorizer.transform(test_tokens)
X_test = X_test.toarray()

#### *Using TF_IDF*

In [26]:
vectorizer2 = TfidfVectorizer(ngram_range=(1,1), stop_words= STOPWORDS)
Z = vectorizer2.fit_transform(tokens)

In [27]:
categories = df_train.Category.values

## *Classification Method:* Support Vector Machine

In [28]:
# parameters = {'kernel':('linear', 'rbf'), 'C':[1.0], 'gamma':'auto'}

clf = svm.SVC(kernel = 'linear')
clf.fit(X, y = categories)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [29]:
print(clf.predict(X_test[127].reshape(1, -1)))

['sport']


## *Classification Method:* Naive Bayes
### *Multinomial NB*

In [30]:
clf = MultinomialNB()
clf.fit(X, y = categories)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [33]:
print(clf.predict(X_test[301].reshape(1,-1)))

['business']


### *Cross Validation*

In [81]:
score_metrics = ['f1_macro', 'precision_macro','recall_macro', 'accuracy']

In [82]:
validation = cross_validate(estimator = clf, X = X, y = categories, scoring = score_metrics, cv = 10, n_jobs = 6)

In [83]:
validation

{'fit_time': array([1.97099042, 2.66030192, 1.93930149, 2.39940977, 2.71562696,
        2.57208204, 1.5123539 , 1.36392975, 1.23693156, 1.16146564]),
 'score_time': array([0.02993131, 0.05465436, 0.03096032, 0.03028154, 0.06327748,
        0.05431747, 0.02559114, 0.02994704, 0.02082372, 0.01963162]),
 'test_f1_macro': array([0.958264  , 0.98847391, 0.97170548, 0.97796785, 0.97078313,
        0.99454579, 0.97732065, 0.98897433, 0.98342675, 0.95789029]),
 'test_precision_macro': array([0.95881773, 0.98857143, 0.97166833, 0.97704648, 0.97161005,
        0.99411765, 0.97708333, 0.99069767, 0.98388889, 0.9615444 ]),
 'test_recall_macro': array([0.95976101, 0.98887195, 0.97285363, 0.97930525, 0.97029913,
        0.99512195, 0.9779896 , 0.98766603, 0.98345528, 0.9583112 ]),
 'test_accuracy': array([0.96067416, 0.98876404, 0.97191011, 0.97752809, 0.97191011,
        0.99438202, 0.97752809, 0.98876404, 0.98314607, 0.96067416])}